<a href="https://colab.research.google.com/github/programminghistorian/ph-submissions/blob/gh-pages/en/drafts/originals/facial_recognition_ai_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Preliminaries

In [ ]:
%mkdir yearbook
%cd yearbook
!pip install --upgrade --no-cache-dir gdown
!gdown --id "1NHT8NN8ClBEnUC5VqkP3wr2KhyiIQzyU"
!unzip PHfiles.zip
%mkdir images
!pip install PyMuPDF
!pip install dlib
!pip install DeepFace
import os, shutil, fitz, cv2, numpy as np, pandas as pd, dlib, tensorflow as tf
from os.path import dirname, join
from deepface import DeepFace

/content/yearbook
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.4
    Uninstalling gdown-4.6.4:
      Successfully uninstalled gdown-4.6.4
/usr/local/lib/python3.9/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1NHT8NN8ClBEnUC5VqkP3wr2KhyiIQzyU
To: /content/yearbook/PHfiles.zip
100% 185M/185M [00:06<00:00, 26.5MB/s]
Archive:  PHfiles.zip
  inflating: 1911.pdf                
  inflating: 1921.pdf                
  inflating: 1931.pdf                
  inflating: 1941.pdf                
  inflating: 1951.pdf                
  inflating: 1961.pdf                
  inflating: haarcascade_frontalface_default.xml  
Looking in indexes: https://pypi.org/simple, https://us-pyth

##PDF Conversion

In [ ]:
path = r'./'
pdfs = [f for f in os.listdir(path) if f.endswith('.pdf')]
for pdf in pdfs:
    os.chdir(os.path.join('./images'))
    os.mkdir((pdf.split(".")[0]))
    newdir = (os.path.join('./images/' + os.path.join(pdf.split(".")[0])))
    os.chdir("..")
    print ("Now copying images into " + (newdir))
    shutil.copy(pdf, newdir)
    os.chdir(newdir)
    doc = fitz.open(pdf)
    for page in doc:
      pix = page.get_pixmap()
      pix.save("page-%i.png" % page.number)
    os.chdir(os.path.dirname(os.getcwd()))
    os.chdir("..")



Now copying images into ./images/1961
Now copying images into ./images/1951
Now copying images into ./images/1911
Now copying images into ./images/1941
Now copying images into ./images/1931
Now copying images into ./images/1921


##Object Detection and Facial Recognition: Code

In [ ]:
path = r'./'

os.chdir(os.path.join(path + 'images'))
dirs = os.listdir(path)
for dir in dirs:
    os.chdir(os.path.join(path + dir))
    pngs = [f for f in os.listdir(path) if f.endswith('.png')]

    if not os.path.exists((dir) + ' faces'):
        print("New 'faces' directory created in " + (dir) + " folder")
        os.makedirs((dir) + ' faces')

        count = 0
        for png in pngs:
            image = cv2.imread(png)

            greyscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

            detected_faces = face_cascade.detectMultiScale(image=greyscale_image, scaleFactor=1.9, minNeighbors=4)

            count = 0
            for (x,y,w,h) in detected_faces:
                try:
                    xpadding = 20
                    ypadding = 40
                    crop_face = image[y-ypadding: y + h+ypadding, x-xpadding: x + w+xpadding]
                    count+=1
                    face = cv2.rectangle(crop_face,(x,y),(x+w,y+h),(255,0,0),2)
                    cv2.imwrite(path + (dir) + ' faces/' + str(count) + '_' + png, face)
                except (Exception):
                    print("An error happened")
                    continue
            os.remove(os.path.join(path, png))
    os.chdir("..")


New 'faces' directory created in 1921 folder
New 'faces' directory created in 1911 folder
New 'faces' directory created in 1931 folder
New 'faces' directory created in 1961 folder
New 'faces' directory created in 1941 folder
New 'faces' directory created in 1951 folder


##Identify Smiles: Code

In [ ]:
%cd ..
numberSmiles = 0
smileCounts = []
numberNonSmiles = 0
nonSmileCounts = []
numErrors = 0
errorCounts = []
pngs = []

fileCount = 0
fileCountList = []

years = ['1911', '1921', '1931', '1941', '1951', '1961']

for year in years:
    path = r'./images' + '/' + year
    for root, dirs, files in os.walk(path):
        for dir in dirs:
            path = path + '/' + (year + ' faces')
            if(fileCount != 0):
                fileCountList.append(fileCount)
            fileCount = 0
            for f in os.listdir(path):
                if f.endswith('.png'):
                    pngs.append(path + '/' + f)
                    fileCount = fileCount + 1

fileCountList.append(fileCount)

totalLoops = 0
count = 0
iterator = 0
for png in pngs:
    try:
        totalLoops = totalLoops + 1
        count = count + 1
        if(count != (fileCountList[iterator] + 1)):
            demography = DeepFace.analyze(png, actions = ['emotion'])
            print(demography)
            if(demography[0]['dominant_emotion'] == 'happy'):
                numberSmiles = numberSmiles + 1
            else:
                numberNonSmiles = numberNonSmiles + 1

        else:
            count = count - 1
            smileCounts.append(numberSmiles / count)
            nonSmileCounts.append(numberNonSmiles / count)
            errorCounts.append(numErrors / count)
            numberSmiles = 0
            numberNonSmiles = 0
            numErrors = 0
            iterator = iterator + 1
            count = 0

    except (Exception):
        numErrors = numErrors + 1
        print("An error happened")
        continue

smileCounts.append(numberSmiles / count)
nonSmileCounts.append(numberNonSmiles / count)
errorCounts.append(numErrors / count)

dict = {'Years': years, 'Smiles': smileCounts, 'NonSmiles': nonSmileCounts, "Error Weight": errorCounts}
data = pd.DataFrame(dict)
data.to_csv('YearbookOutput.csv', index=False)
print(count)


/content/yearbook
facial_expression_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /root/.deepface/weights/facial_expression_model_weights.h5
100%|██████████| 5.98M/5.98M [00:00<00:00, 115MB/s]
Action: emotion: 100%|██████████| 1/1 [00:06<00:00,  6.58s/it]


[{'emotion': {'angry': 2.7204811573028564, 'disgust': 3.8496024146184027e-07, 'fear': 1.0063357651233673, 'happy': 0.00680559896863997, 'sad': 10.03282368183136, 'surprise': 0.0020465062334551476, 'neutral': 86.23150587081909}, 'dominant_emotion': 'neutral', 'region': {'x': 30, 'y': 50, 'w': 65, 'h': 65}}]


Action: emotion: 100%|██████████| 1/1 [00:00<00:00, 12.54it/s]


[{'emotion': {'angry': 2.1246447489089073, 'disgust': 1.901664848713893e-08, 'fear': 0.034286557916850396, 'happy': 0.0002512492229182152, 'sad': 4.2414514879642145, 'surprise': 1.7301296109425963e-07, 'neutral': 93.59936194427468}, 'dominant_emotion': 'neutral', 'region': {'x': 28, 'y': 51, 'w': 63, 'h': 63}}]


...

[{'emotion': {'angry': 99.99678730945512, 'disgust': 5.207611809676494e-14, 'fear': 0.0006675768361222395, 'happy': 2.0377734049083237e-08, 'sad': 0.002541819200433259, 'surprise': 2.750607197756882e-10, 'neutral': 3.446654012241122e-06}, 'dominant_emotion': 'angry', 'region': {'x': 24, 'y': 41, 'w': 80, 'h': 80}}]


Action: emotion: 100%|██████████| 1/1 [00:00<00:00, 11.48it/s]


[{'emotion': {'angry': 0.0002536472493375186, 'disgust': 1.9477794193095948e-22, 'fear': 2.944826260005639e-16, 'happy': 99.9984622001648, 'sad': 7.778225112609061e-10, 'surprise': 2.221436061359064e-11, 'neutral': 0.0012850216080551036}, 'dominant_emotion': 'happy', 'region': {'x': 18, 'y': 38, 'w': 90, 'h': 90}}]


Action: emotion: 100%|██████████| 1/1 [00:00<00:00,  9.16it/s]

[{'emotion': {'angry': 5.050151709182721e-08, 'disgust': 1.1292334353525661e-16, 'fear': 8.108859702972249e-05, 'happy': 99.52624437369373, 'sad': 1.429298453742152e-06, 'surprise': 0.008675461657326246, 'neutral': 0.46499658353606366}, 'dominant_emotion': 'happy', 'region': {'x': 25, 'y': 44, 'w': 77, 'h': 77}}]
57


##Download and results

In [ ]:
from google.colab import files
files.download('YearbookOutput.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>